# Daily Trends Prediction - LSTM Model

This notebook trains an LSTM (Long Short-Term Memory) deep learning model to predict daily interest values.

**Algorithm**: LSTM Neural Network
- Captures long-term dependencies in time series
- Handles sequential data naturally
- Good for complex patterns

**Target**: Predict interest_value (0-100) for next N days

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

## 2. Load Processed Data

In [ ]:
# Load processed daily trends data
data_path = '../data/processed/daily_trends_processed_latest.csv'

df = pd.read_csv(data_path)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['keyword', 'category', 'date'])

print(f"Data shape: {df.shape}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Keywords: {df['keyword'].nunique()}")

df.head()

## 3. Prepare Sequences for LSTM

In [ ]:
def create_sequences(data, lookback=30, forecast_horizon=7):
    """
    Create sequences for LSTM training
    
    Args:
        data: Time series data
        lookback: Number of past days to use
        forecast_horizon: Number of future days to predict
    
    Returns:
        X, y arrays for training
    """
    X, y = [], []
    
    for i in range(len(data) - lookback - forecast_horizon + 1):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback:i+lookback+forecast_horizon])
    
    return np.array(X), np.array(y)

# Parameters
LOOKBACK = 30  # Use 30 days of history
FORECAST_HORIZON = 7  # Predict next 7 days

print(f"Sequence parameters:")
print(f"  Lookback: {LOOKBACK} days")
print(f"  Forecast horizon: {FORECAST_HORIZON} days")

In [ ]:
# Prepare data for a specific keyword/category
# For simplicity, we'll train one model per keyword-category combination
# In production, you might want to train a single model for all

sample_keyword = df['keyword'].iloc[0]
sample_category = df['category'].iloc[0]

# Filter data
keyword_data = df[
    (df['keyword'] == sample_keyword) & 
    (df['category'] == sample_category)
].copy()

keyword_data = keyword_data.sort_values('date')

print(f"Training model for: {sample_keyword} ({sample_category})")
print(f"Data points: {len(keyword_data)}")

# Extract interest values
values = keyword_data['interest_value'].values.reshape(-1, 1)

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values)

print(f"Data normalized to range [0, 1]")

In [ ]:
# Create sequences
X, y = create_sequences(scaled_values, LOOKBACK, FORECAST_HORIZON)

print(f"Sequence shapes:")
print(f"  X: {X.shape} (samples, lookback, features)")
print(f"  y: {y.shape} (samples, forecast_horizon, features)")

# Train/test split (80/20)
split_idx = int(len(X) * 0.8)

X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"\nTrain set: {X_train.shape[0]} sequences")
print(f"Test set: {X_test.shape[0]} sequences")

## 4. Build LSTM Model

In [ ]:
def build_lstm_model(lookback, forecast_horizon):
    """
    Build LSTM model architecture
    """
    model = Sequential([
        # First LSTM layer
        LSTM(128, return_sequences=True, input_shape=(lookback, 1)),
        Dropout(0.2),
        
        # Second LSTM layer
        LSTM(64, return_sequences=False),
        Dropout(0.2),
        
        # Dense layers
        Dense(32, activation='relu'),
        Dropout(0.2),
        
        # Output layer
        Dense(forecast_horizon)
    ])
    
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Build model
model = build_lstm_model(LOOKBACK, FORECAST_HORIZON)

print("Model Architecture:")
model.summary()

## 5. Train Model

In [ ]:
# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    '../models/lstm_daily_model.keras',
    monitor='val_loss',
    save_best_only=True
)

# Reshape y for training (flatten forecast horizon)
y_train_flat = y_train.reshape(y_train.shape[0], -1)
y_test_flat = y_test.reshape(y_test.shape[0], -1)

# Train model
print("Training LSTM model...")
history = model.fit(
    X_train, y_train_flat,
    validation_data=(X_test, y_test_flat),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

print("\nModel training completed")

## 6. Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Model Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='Train MAE')
axes[1].plot(history.history['val_mae'], label='Val MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Model MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Model Evaluation

In [ ]:
# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Inverse transform predictions
y_train_inv = scaler.inverse_transform(y_train_flat)
y_test_inv = scaler.inverse_transform(y_test_flat)
y_pred_train_inv = scaler.inverse_transform(y_pred_train)
y_pred_test_inv = scaler.inverse_transform(y_pred_test)

# Calculate metrics
def calculate_metrics(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{dataset_name} Metrics:")
    print(f"  MAE:  {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R2:   {r2:.4f}")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

train_metrics = calculate_metrics(y_train_inv, y_pred_train_inv, "Train")
test_metrics = calculate_metrics(y_test_inv, y_pred_test_inv, "Test")

## 8. Visualize Predictions

In [ ]:
# Plot predictions for test set
# Show first 5 test sequences
n_samples = min(5, len(y_test_inv))

fig, axes = plt.subplots(n_samples, 1, figsize=(14, 3*n_samples))
if n_samples == 1:
    axes = [axes]

for i in range(n_samples):
    axes[i].plot(range(FORECAST_HORIZON), y_test_inv[i], 
                label='Actual', marker='o', linewidth=2)
    axes[i].plot(range(FORECAST_HORIZON), y_pred_test_inv[i], 
                label='Predicted', marker='s', linewidth=2, alpha=0.7)
    axes[i].set_title(f'Test Sample {i+1}')
    axes[i].set_xlabel('Days Ahead')
    axes[i].set_ylabel('Interest Value')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot: Actual vs Predicted (all forecast horizons)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Train set
axes[0].scatter(y_train_inv.flatten(), y_pred_train_inv.flatten(), alpha=0.3, s=10)
axes[0].plot([0, 100], [0, 100], 'r--', lw=2)
axes[0].set_xlabel('Actual Interest Value')
axes[0].set_ylabel('Predicted Interest Value')
axes[0].set_title(f'Train Set (R2 = {train_metrics["R2"]:.4f})')
axes[0].grid(True, alpha=0.3)

# Test set
axes[1].scatter(y_test_inv.flatten(), y_pred_test_inv.flatten(), alpha=0.3, s=10, color='orange')
axes[1].plot([0, 100], [0, 100], 'r--', lw=2)
axes[1].set_xlabel('Actual Interest Value')
axes[1].set_ylabel('Predicted Interest Value')
axes[1].set_title(f'Test Set (R2 = {test_metrics["R2"]:.4f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Save Model and Artifacts

In [ ]:
# Model is already saved via ModelCheckpoint callback
print(f"Model saved to: ../models/lstm_daily_model.keras")

# Save scaler
import joblib
joblib.dump(scaler, '../models/lstm_scaler.pkl')
print(f"Scaler saved to: ../models/lstm_scaler.pkl")

# Save metrics
import json
metrics = {
    'train': train_metrics,
    'test': test_metrics,
    'lookback': LOOKBACK,
    'forecast_horizon': FORECAST_HORIZON,
    'keyword': sample_keyword,
    'category': sample_category
}

with open('../models/lstm_daily_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved to: ../models/lstm_daily_metrics.json")

## 10. Summary

### Model Performance:
- Algorithm: LSTM Neural Network
- Lookback: 30 days
- Forecast Horizon: 7 days
- Test RMSE: Check output above
- Test R2: Check output above

### Advantages:
- Captures long-term dependencies
- Handles sequential patterns well
- Multi-step forecasting

### Limitations:
- Requires more data than traditional ML
- Longer training time
- Needs careful hyperparameter tuning

### Next Steps:
1. Try different architectures (GRU, Bidirectional LSTM)
2. Add attention mechanism
3. Experiment with different lookback windows
4. Train separate models for each keyword/category